# Stochastic Variational Method with RL algorithms

In [1]:
import numpy as np
import gym
import torch
import subprocess
import os
import pickle

## Expoloring environment

In [2]:
env = gym.make('svm_env:svmEnv-v2', n_pairs = 3, n_basis = 50, file_sigmas ="./svmCodeSVD/sigmas.dat" )

print('### Env Name ######', env.unwrapped.spec.id)

obs_space = env.observation_space

print('###### Observation space ####### \n', obs_space)

state_size = env.observation_space.shape[-1]

print('###### Size of observation space ####### \n', state_size)

act_space = env.action_space.shape

print('###### Action space ####### \n', act_space)

act_size = env.action_space.shape[0]*env.action_space.shape[-1]

print('###### Number of actions ####### \n', act_size)

state = env.reset()

print('##### State after reset ###### \n', state)

print('##### File where will be stored sigmas \n', env.file_sigmas)



### Env Name ###### svmEnv-v2
###### Observation space ####### 
 Box(-inf, inf, (1,), float32)
###### Size of observation space ####### 
 1
###### Action space ####### 
 (50, 3)
###### Number of actions ####### 
 150
*****CALL RESET******
Action chosen at reset:  [0.]
##### State after reset ###### 
 [0.]
##### File where will be stored sigmas 
 ./svmCodeSVD/sigmas.dat


# Your codes `DDPG` and `PPO`

## Functions for saving and clean

In [3]:
## Save all rewards, energies and princip dims in files during episode training
def create_run_fold_and_info(agent, env):
    
    # Check if folder exist and creat it
    i = 0
    while os.path.exists(f'runs_optim_envs/run_{i}/'):
        i += 1
    name_dir = f'runs_optim_envs/run_{i}/'
    os.makedirs(name_dir)
    
    # Create info.p to store info in pickle file
    info = {'alg':agent.name, 'env':env.unwrapped.spec.id \
            , 'batch_size':agent.batch_size, 'bootstrap_size':agent.bootstrap_size \
            , 'gamma':agent.gamma, 'tau':agent.tau,'lr_critic':agent.lr_critic \
            , 'lr_actor':agent.lr_actor, 'update_every':agent.update_every \
            , 'transfer_every':agent.transfer_every, 'num_update':agent.num_update \
            , 'add_noise_every':agent.add_noise_every}
    
    pickle.dump(info, open(name_dir+'info.p', 'wb'))
    return name_dir
    
def save_all(name_run_dir, i_ep, sigmas_i_ep, rew_i_ep, en_i_ep, pri_dim_i_ep \
             , full_dim_i_ep, act_model_i_ep, cr_model_i_ep):
    
    pickle.dump(sigmas_i_ep, open(name_run_dir+f'sigmas_{i_ep}.p', 'wb'))
    pickle.dump(rew_i_ep, open(name_run_dir+f'rew_{i_ep}.p', 'wb'))
    pickle.dump(en_i_ep, open(name_run_dir+f'en_{i_ep}.p', 'wb'))
    pickle.dump(pri_dim_i_ep, open(name_run_dir+f'pri_dim_{i_ep}.p', 'wb'))
    pickle.dump(full_dim_i_ep, open(name_run_dir+f'full_dim_{i_ep}.p', 'wb'))
    pickle.dump(act_model_i_ep, open(name_run_dir+f'act_model_{i_ep}.p', 'wb'))
    pickle.dump(cr_model_i_ep, open(name_run_dir+f'cr_model_{i_ep}.p', 'wb'))
    
def rm_useless_file(actor_model_file, critic_model_file, file_sigmas):
    os.remove(actor_model_file)
    os.remove(critic_model_file)
    os.remove(file_sigmas)

## From my `ddpg_agent.py` code

In [4]:
from ddpg_agent import DDPG_agent
agent = DDPG_agent(state_size, act_size, seed = 0)

In [5]:
## Run ddpg algs   
def run_ddpg(max_t_step = 10, n_episodes=10):
    
    # Create h5 file and store info about alg and its hypereparams
    name_run_dir = create_run_fold_and_info(agent, env)
    
    for i_ep in range(n_episodes):
        state = env.reset()
        agent.reset()
        rew_i_ep = []
        en_i_ep = []
        pri_dim_i_ep = []
        full_dim_i_ep = []
        action_i_episode = []

        ## Training loop of each episode
        for t_step in range(max_t_step):
            action = agent.act(state)
            next_state, reward, done, info = env.step(action.reshape((env.n_basis,env.n_pairs)))
            agent.step(state, action, reward, next_state, done)
            state = next_state

            # Save rew, energies, princip dims, act and crit models
            action_i_episode.append(action.reshape((env.n_basis,env.n_pairs)))
            rew_i_ep.append(reward)
            en_i_ep.append(state[0])
            pri_dim_i_ep.append(env.princp_dim)
            full_dim_i_ep.append(env.full_dim)
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            if done:
                break
                
        ## Save data during training (to not lose the work done)
        save_all(name_run_dir=name_run_dir, i_ep=int(i_ep), sigmas_i_ep=action_i_episode \
                 , rew_i_ep=rew_i_ep, en_i_ep=en_i_ep, pri_dim_i_ep=pri_dim_i_ep \
                 , full_dim_i_ep=full_dim_i_ep, act_model_i_ep='checkpoint_actor.pth' \
                 , cr_model_i_ep='checkpoint_critic.pth')
        
        print('Episode {} ... Score: {:.3f}'.format(i_ep, np.sum(rew_i_ep)))

    rm_useless_file('checkpoint_actor.pth', 'checkpoint_critic.pth', env.file_sigmas)
    return name_run_dir

In [6]:
all_data = run_ddpg(10, 10)

*****CALL RESET******
Action chosen at reset:  [0.]
****CALL STEP****
Action chosen at step:  [[41.326897 68.98927  55.587734]
 [48.889088 59.22312  35.692894]
 [51.29618  57.28906  48.881516]
 [41.43883  50.13721  62.284298]
 [89.4814   59.430107 53.87242 ]
 [56.14401  50.503357 44.48779 ]
 [51.86254  58.857254 57.10333 ]
 [46.87075  50.75939  40.84858 ]
 [58.652058 39.463802 53.874794]
 [54.72371  58.052185 58.337666]
 [54.806976 64.17156  84.03984 ]
 [70.155266 52.255135 66.979485]
 [54.24046  59.21776  57.1173  ]
 [48.28026  55.15761  36.95992 ]
 [61.50089  56.796158 36.627068]
 [37.003384 73.59915  59.882336]
 [67.41905  59.715607 85.764145]
 [49.90651  71.252785 59.29526 ]
 [57.004475 64.53133  69.82104 ]
 [58.951782 28.688417 52.747124]
 [53.03532  72.84357  58.07345 ]
 [62.052658 45.58359  69.56333 ]
 [61.38929  56.885548 51.68843 ]
 [18.923637 63.8274   54.944267]
 [57.244347 38.59173  36.41712 ]
 [43.869205 46.660786 65.255905]
 [49.579334 50.732285 73.27075 ]
 [68.653984 58.

With this action the energy is:  -0.134066
With this action the full dim is:  50  and princip dim is:  49
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.134066
Set reward :  8.862341329415479
****CALL STEP****
Action chosen at step:  [[ 33.679382   66.66064    59.12148  ]
 [ 24.500326   58.053566   32.26056  ]
 [ 53.85709    65.19148    44.77108  ]
 [ 37.144707   45.222237   33.08874  ]
 [ 73.137474   64.54739    59.945244 ]
 [ 63.31408    64.900665   39.681656 ]
 [ 48.133488   37.39016    92.2818   ]
 [ 72.07161    52.177254   87.973236 ]
 [ 20.943668   21.730762   53.030342 ]
 [ 68.62994    41.77978    17.557968 ]
 [ 85.63463    92.864456   55.463245 ]
 [ 38.66725    90.210266   64.20435  ]
 [ 66.1895     15.745331   66.806305 ]
 [ 72.292786   64.701355   54.247196 ]
 [ 37.671204   57.336967   56.462574 ]
 [ 31.525873   42.355854   84.927574 ]
 [ 71.28471    27.636143   59.134293 ]
 [ 61.9612     53.3092     74.703674 ]
 [ 38.277763   60.66115    25.126

With this action the energy is:  -0.100199
With this action the full dim is:  50  and princip dim is:  49
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.100199
Set reward :  6.507248315602073
****CALL STEP****
Action chosen at step:  [[ 34.004272   79.83817    50.273636 ]
 [ 46.65849    41.534424   73.62589  ]
 [ 63.144726  102.232185   27.460356 ]
 [ 61.95534    56.23026     6.0188675]
 [ 75.10234    39.630314   76.04285  ]
 [ 57.90514    87.71609    46.07244  ]
 [ 60.965282   57.806057   63.88155  ]
 [ 54.27757    25.75589    84.58429  ]
 [ 32.966843   33.09056    56.529137 ]
 [ 86.04549     7.004551   15.423931 ]
 [ 97.09453    86.08309    65.730675 ]
 [ 33.882755   59.887028   25.408464 ]
 [ 80.72923    26.941484   87.34805  ]
 [ 49.87752    57.458027   53.895905 ]
 [ 30.886972   82.24356    60.090816 ]
 [ 40.50428    39.98011    36.003975 ]
 [ 54.641026    3.6561623  51.06532  ]
 [ 73.66351    66.42173    60.41052  ]
 [ 40.29087    61.323986   28.005

With this action the energy is:  -0.0840137
With this action the full dim is:  50  and princip dim is:  48
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0840137
Set reward :  5.381731235362485
****CALL STEP****
Action chosen at step:  [[ 39.69971    57.616013   49.34289  ]
 [ 32.384903   47.775513   36.855553 ]
 [ 72.2301     78.84552     5.522255 ]
 [ 43.749275   60.321194    8.468388 ]
 [ 35.363777   57.9575     77.98147  ]
 [ 81.40776    84.036705   39.92944  ]
 [ 20.804089   63.591843   47.40631  ]
 [ 68.83162    58.178932   47.751305 ]
 [ 57.702953    2.9481812  71.34408  ]
 [ 60.250572   60.62394    47.847675 ]
 [ 86.67345    31.022291   67.62132  ]
 [ 64.652374   45.480156   87.08812  ]
 [ 41.488976   38.680733   12.469711 ]
 [ 43.5062     66.22968    64.37441  ]
 [ 26.655561   41.766624   50.520508 ]
 [ 66.98982    44.10478    44.120834 ]
 [ 63.678253   52.97131    38.988277 ]
 [ 76.71607    29.568844   77.94588  ]
 [ 52.583126   43.04497    48.1

With this action the energy is:  -0.111585
With this action the full dim is:  50  and princip dim is:  48
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.111585
Set reward :  7.2990246481759495
****CALL STEP****
Action chosen at step:  [[48.763897  58.981964  88.441284 ]
 [54.08146   62.254696  38.532295 ]
 [52.29311   58.33161   30.01917  ]
 [25.805685  78.927605   1.       ]
 [40.89472   51.360126  31.9734   ]
 [93.01083   68.14388   38.111908 ]
 [24.629925  53.27687   51.367466 ]
 [66.39863   59.777245  45.32629  ]
 [55.234936  33.303078  38.973167 ]
 [50.78676   81.24705   61.95667  ]
 [57.59647   36.133976  57.18177  ]
 [72.62708   53.92046   88.86922  ]
 [69.34933   41.505478  38.351624 ]
 [48.891014  54.337753  75.82645  ]
 [42.336357  41.82577   68.16199  ]
 [49.533257  42.445145  24.452116 ]
 [72.126465  26.804136  57.788    ]
 [73.13686   36.875427  63.203224 ]
 [45.60701    4.398304  29.712269 ]
 [38.543854  39.79442   29.34102  ]
 [19.055534  5

...

With this action the energy is:  -0.0488859
With this action the full dim is:  50  and princip dim is:  44
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0488859
Set reward :  2.938962865839512
****CALL STEP****
Action chosen at step:  [[28.190466 62.47266  47.523197]
 [68.93037  40.07841  32.003468]
 [63.832737 64.68102  40.436283]
 [46.418266 66.32342  46.67482 ]
 [54.49462  26.426254 77.6977  ]
 [57.220818 28.18444  51.225056]
 [42.051678 63.858547 69.995926]
 [22.234211 76.54282  61.19825 ]
 [39.15943  53.059692 49.47113 ]
 [59.384068 47.582172 64.60004 ]
 [71.23276  70.4725   83.11413 ]
 [76.9083   56.039505 53.75403 ]
 [60.384293 29.222479 76.46565 ]
 [64.14649  39.772446 55.622032]
 [48.819565 66.73039  59.07714 ]
 [61.49665  25.289162 42.104366]
 [54.178932 25.516556 57.14176 ]
 [74.39986  55.280575 71.56169 ]
 [89.42616  68.4733   57.93313 ]
 [36.392353 64.71904  54.677895]
 [38.989635 68.02301  66.23338 ]
 [47.243675 38.831577 98.62596 ]
 [78.02

With this action the energy is:  -0.073912
With this action the full dim is:  50  and princip dim is:  50
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.073912
Set reward :  4.679264435892828
****CALL STEP****
Action chosen at step:  [[ 28.119684   41.485157   50.154907 ]
 [ 66.24278    49.350956   23.917515 ]
 [ 86.778854   70.97293    20.642303 ]
 [ 34.303814   39.053642   54.733143 ]
 [ 66.06994    32.619278   67.33439  ]
 [ 49.629215   59.06801    22.422718 ]
 [ 50.13341    64.03717    67.23717  ]
 [ 24.20792    70.98085    74.08486  ]
 [ 52.67643    34.5045     35.097153 ]
 [ 63.94199    56.533855   48.935482 ]
 [ 26.621037   57.724022   99.02901  ]
 [ 90.97135    54.58726    56.238388 ]
 [ 36.274567   43.403305   87.07086  ]
 [ 56.18124    45.94909    66.79956  ]
 [ 55.447746   60.200863   12.817383 ]
 [ 69.70001    46.705284   44.606335 ]
 [ 72.13331    43.945797   58.33441  ]
 [ 55.16808    78.137184   71.71872  ]
 [ 68.848564   96.024704   63.405

With this action the energy is:  -0.0844021
With this action the full dim is:  50  and princip dim is:  50
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0844021
Set reward :  5.4087403630255535
****CALL STEP****
Action chosen at step:  [[ 59.112854   53.0694     71.276344 ]
 [ 63.88912    56.510498   27.030624 ]
 [ 57.99853    86.80476     1.       ]
 [ 50.216732   34.10731    72.05094  ]
 [ 29.296837   34.41693    41.1608   ]
 [ 17.531223   82.73757    56.9132   ]
 [ 60.89554    54.594177   72.43455  ]
 [ 20.394035   85.00153    74.50687  ]
 [ 25.984968   10.907394   22.214165 ]
 [ 73.23358    47.037575   63.66427  ]
 [ 14.228958   66.58948    92.89601  ]
 [104.83606    76.47728    74.95509  ]
 [ 38.576294   75.61326    94.82745  ]
 [ 36.970066   27.121014   57.618233 ]
 [ 28.837124   49.406708   22.396587 ]
 [ 21.827682   56.704567   45.76152  ]
 [ 82.39631    53.736633   74.565895 ]
 [ 49.626884   79.19259    76.36945  ]
 [ 38.780945   53.673183   78.

With this action the energy is:  -0.0839498
With this action the full dim is:  50  and princip dim is:  47
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0839498
Set reward :  5.37728766363831
****CALL STEP****
Action chosen at step:  [[ 66.67254    50.548824   38.54698  ]
 [ 48.934826   24.02734    61.800735 ]
 [ 67.14026    51.646347   44.644524 ]
 [ 46.982754   26.090578   63.565567 ]
 [ 25.531065   39.19717    52.651066 ]
 [ 25.81262    59.266      59.347088 ]
 [ 61.431202   51.959976   17.203224 ]
 [  1.         66.266426   39.838997 ]
 [ 85.29955    50.08593    50.349937 ]
 [ 38.447956   58.904274   44.28389  ]
 [ 32.069557   61.380356   71.59207  ]
 [ 53.908222   71.74536    38.176666 ]
 [ 69.530945   11.292469   60.719246 ]
 [ 52.11249    48.171455   69.485794 ]
 [ 72.413185   85.029816   41.759323 ]
 [ 37.358124   92.148254   66.551    ]
 [ 45.51373    66.10988    54.986423 ]
 [ 61.70317    50.348087   82.87063  ]
 [ 73.08905    68.1559     32.96

With this action the energy is:  -0.0841987
With this action the full dim is:  50  and princip dim is:  50
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0841987
Set reward :  5.39459603612888
****CALL STEP****
Action chosen at step:  [[ 50.840775   72.5003     41.917343 ]
 [ 52.172417    1.         52.17785  ]
 [ 69.91957    44.282898   14.129799 ]
 [ 42.38736     9.743992   70.932816 ]
 [ 46.68638    35.228832   27.207285 ]
 [ 19.142715   62.77353    71.55912  ]
 [ 67.1468     61.591835   23.63118  ]
 [ 27.909546  104.07356    69.252426 ]
 [ 94.60664    47.648815   57.897594 ]
 [ 27.008299   67.437126   36.592384 ]
 [ 29.572912   60.409035   86.41126  ]
 [ 68.69773    61.543037   61.319595 ]
 [ 68.83096     3.7768364  81.77573  ]
 [ 56.75248    51.252026   74.07371  ]
 [ 23.69475    81.63413    28.046515 ]
 [  1.         90.325485   67.41424  ]
 [ 59.850834   45.135452   79.78562  ]
 [ 62.56062    73.78535    80.76652  ]
 [ 43.175587   52.466747   18.51

## Random search as in original SVM

In [ ]:
state = env.reset()
scores = []
step = 0
score = 0.0

while True:
    print(".....STEP.....", step)
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    step = step + 1
    score += reward
    scores.append(score)
    state = next_state
    if done:
        break